# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
city_csv = "output_data/cities.csv"
city_data = pd.read_csv(city_csv)
city_data.describe()


,City ID,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,559.000000,559.000000,559.000000,559.000000,559.000000,559.000000,559.000000,5.590000e+02
mean,279.000000,19.715590,13.529192,48.162701,73.245081,52.670841,8.195510,1.612757e+09
std,161.513673,33.593116,90.357738,32.444635,19.197120,39.354137,6.044869,8.204232e+01
min,0.000000,-54.800000,-179.166700,-40.000000,11.000000,0.000000,0.110000,1.612756e+09
25%,139.500000,-7.961400,-66.603550,27.960000,64.000000,12.000000,3.440000,1.612756e+09
50%,279.000000,22.566700,15.793200,59.000000,77.000000,54.000000,6.910000,1.612757e+09
75%,418.500000,48.308300,92.825000,75.030000,88.000000,91.000000,11.020000,1.612757e+09
max,558.000000,78.218600,179.316700,95.000000,100.000000,100.000000,39.120000,1.612757e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [20]:
humidity_data = city_data[["Lat", "Lng","Humidity"]]
locations = humidity_data[["Lat", "Lng"]].astype(float)
weights = humidity_data["Humidity"].astype(float)
# fig_layout = ['width':]

# fig = gmaps.figure(center=(0,0), zoom_level=10)
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3
                                )

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [8]:
ideal_conditions = city_data.loc[(city_data["Max Temp"] >= 70) & 
                                 (city_data["Max Temp"] < 80) & 
                                 (city_data["Wind Speed"] < 10) & 
                                 (city_data["Cloudiness"] == 0)
                                ]
ideal_conditions

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
61,61,lima,-12.0432,-77.0282,75.00,83,0,8.05,PE,1612756493
219,219,maceio,-9.6658,-35.7353,73.40,94,0,3.44,BR,1612756528
264,264,saiha,22.4833,92.9667,77.05,27,0,1.95,IN,1612756536
337,337,natitingou,10.3042,1.3796,75.20,29,0,6.91,BJ,1612756548
472,472,cidreira,-30.1811,-50.2056,70.05,74,0,7.63,BR,1612756570
499,499,akyab,20.1500,92.9000,74.91,40,0,7.18,MM,1612756574
500,500,ballitoville,-29.5390,31.2144,75.00,94,0,1.14,ZA,1612756575
540,540,sanchor,24.7553,71.7722,71.94,21,0,3.76,IN,1612756581
548,548,pereira barreto,-20.6383,-51.1092,70.32,73,0,6.89,BR,1612756505


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
hotel_df = ideal_conditions
hotel_df["Hotel Name"] = ""
hotel_df

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
61,61,lima,-12.0432,-77.0282,75.00,83,0,8.05,PE,1612756493,
219,219,maceio,-9.6658,-35.7353,73.40,94,0,3.44,BR,1612756528,
264,264,saiha,22.4833,92.9667,77.05,27,0,1.95,IN,1612756536,
337,337,natitingou,10.3042,1.3796,75.20,29,0,6.91,BJ,1612756548,
472,472,cidreira,-30.1811,-50.2056,70.05,74,0,7.63,BR,1612756570,
499,499,akyab,20.1500,92.9000,74.91,40,0,7.18,MM,1612756574,
500,500,ballitoville,-29.5390,31.2144,75.00,94,0,1.14,ZA,1612756575,
540,540,sanchor,24.7553,71.7722,71.94,21,0,3.76,IN,1612756581,
548,548,pereira barreto,-20.6383,-51.1092,70.32,73,0,6.89,BR,1612756505,


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
